In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('fede'))
if module_path not in sys.path:
    sys.path.append(module_path)
# import pandas as pd
from client import Client
from supported_modles import Supported_modles
import utils
from fedavg import Fedavg
from sklearn.utils.class_weight import compute_sample_weight
import numpy as np
import random

In [2]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [3]:
selected_model = Supported_modles.NN_classifier

In [4]:
clients = utils.set_data_mock(selected_model, True)

Creating node1.
Creating node2.
Creating node3.
70665
139435
119085
103107
249932
9180
DOWNSAMLING node3


In [5]:
# for client in clients:
#     client.init_empty_model(epochs=50)
#     client.train_model(epochs=50)
#     score = client.test_model_f1()
#     print(score)

In [24]:
for client in clients:
    client.init_empty_model(epochs=50)
    data = client.fed_avg_prepare_data(epochs=10)
    score = client.test_model_f1()
    print(score)

0.9328640431347587
0.8814088754847049
0.9372570794003332


In [50]:
clients[0].login_api()
data = clients[0].fed_avg_prepare_data(epochs=10)
clients[0].send_local_model(data)

loged in succesfully!
200
{"message":"Model created"}



In [51]:
clients[1].login_api()
data = clients[1].fed_avg_prepare_data(epochs=10)
clients[1].send_local_model(data)

loged in succesfully!
200
{"message":"Model created"}



In [52]:
clients[2].login_api()
data = clients[2].fed_avg_prepare_data(epochs=10)
clients[2].send_local_model(data)

loged in succesfully!
200
{"message":"Model created"}



In [54]:
for client in clients:
    client.login_api()
    client.get_global_model()
    score = client.test_model_f1()
    print(score)
    print(client.model.fc3.bias)

loged in succesfully!
0.7945866475666302
Parameter containing:
tensor([-9.0175e-05,  9.0175e-05], requires_grad=True)
loged in succesfully!
0.6597278543750563
Parameter containing:
tensor([-9.0175e-05,  9.0175e-05], requires_grad=True)
loged in succesfully!
0.9333333333333333
Parameter containing:
tensor([-9.0175e-05,  9.0175e-05], requires_grad=True)
